Для работы с private переменными мы начились создавать геттеры, сеттеры и делитеры\
Но хочется все также обращаться через точку именно к balance (приватному атрибуту) и при этом быть безнаказанным.\
И это на самом деле можно устроить с помощью property\
Property это определнный класс, который позволят добиться такого поведения\
Как же это сделать ? Ответ: Нужно создать атрибут класса, который является экземпляром класса property, и назвать атрибут класса так, как вы желаете обращаться к нему через точку.\
В нашем случае имя будет balance (но может быть и другим, например my_balance). Такой атрибут и называется свойством\
\
Итак, property (свойство) - атрибут класса, который является экземпляром класса property\
\
Давайте же теперь научимся устанавливать геттеры, сеттеры и делитеры с помощью декоратора property

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    def get_balance(self): # getter
        print("get_balance")
        return self.__balance

    def set_balance(self, value): # setter
        if not isinstance(value, (int, float)):
            raise ValueError("Balance must be a figure")
        self.__balance = value

    def delete_balance(self): # deleter
        print("delete balance")
        del self.__balance

    balance = property(fget=get_balance,
                       fset=set_balance,
                       fdel=delete_balance)

user = BankAccount("Misha", 100000)
print(user.balance) # 100000
user.balance = 3000
print(user.balance) # 3000
user.balance = "Hello" # ValueError: Balance must be a figure
del user.balance
print(user.balance) # AttributeError

getter и setter мы определили таким образом, что при каждом их вызове срабатывает print.\
Это было сделано с целью показать, в какой момент времени они будут срабатывать.\
С помощью property мы также смогли определить удаление приватного атрибута balance\
\
Поймем что же такое property

In [2]:
x = property()
print(x)
print(type(x))

<class 'property'>


Видим, что x это экземпляр класса property и соответственно type(x) говорит, что у x класс property\
Написав x. через подсказку редактора кода мы можем видеть, что с объектом x можно совершать такие вызовы через точку: x.getter(), x.setter(), x.deleter()\
Результатом вызова всех этих трех методов будет являться объекты property

In [3]:
print(x.getter(10)) # property obj

В эти методы в качестве аргументов мы будем передавать, конечно, не числа, а функции.\
\
Используем полученное знание, а также подчеркнем то, что в качестве имени атрибута, в котором лежит property может быть любым - назовем теперь не balance, а my_balance:

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    def get_balance(self):
        print("get balance")
        return self.__balance

    def set_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

    def delete_balance(self):
        print("delete balance")
        del self.__balance

    my_balance = property()
    my_balance = my_balance.getter(get_balance)
    my_balance = my_balance.setter(set_balance)
    my_balance = my_balance.deleter(delete_balance)

user = BankAccount("Mikhail", 1000)
print(user.my_balance)
user.my_balance = 1234
del user.my_balance
print(user.my_balance) # AttributeError

Также мы можем передать геттер, сеттер и делитер в конструктор property сразу

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    def get_balance(self):
        print("get balance")
        return self.__balance

    def set_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

    def delete_balance(self):
        print("delete balance")
        del self.__balance

    my_balance = property(get_balance, set_balance, delete_balance)

user = BankAccount("Mikhail", 1000)
print(user.my_balance)
user.my_balance = 1234
del user.my_balance
print(user.my_balance) # AttributeError

Таким образом мы рассмотрели три способа задать property в нашем классе
1)    balance = property(fget=get_balance,
                       fset=set_balance,
                       fdel=delete_balance)

2)    my_balance = property()\
      my_balance = my_balance.getter(get_balance)\
      my_balance = my_balance.setter(set_balance)\
      my_balance = my_balance.deleter(delete_balance)\

3)    my_balance = property(get_balance, set_balance, delete_balance)

Хорошо, идем дальше. У нас возникает такая небольшая проблемка - появляется двойная функциональность.\
Под двойной функциональностью имеется в виду следующее:\
Получить значение атрибута __balance мы можем двумя путями:
1) user.get_balance() 
2) user.my_balance\
Как от этого избавиться ? Ответ: использовать декораторы\
\
Для начала дадим геттеру такое же имя как и атрибуту, в котором лежит property. И передадим конструктору property геттер

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    def my_balance(self):
        print("get balance")
        return self.__balance


    my_balance = property(my_balance)

user = BankAccount("Mikhail", 1000)
print(user.my_balance)

Строка my_balance = property(my_balance) есть ни что иное, как использование декоратора property.\
Декоратор это же функция, но property() это конструктор класса. Получается, что функция \_\_init\_\_ в propety это декоратор ?\
Ответа пока что я сам не знаю.\
\
Раз мы использовали декоратор, то как принято у всех, нужно пользоваться синтаксическим сахаром

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

user = BankAccount("Mikhail", 1000)
print(user.my_balance)

Все работает ! Отлично, идем дальше.\
\
Пока что мы разобрались только с геттером, а как же сеттер и тем более делитер ?\
\
my_balance после декорирования стала функцией, которая возвращается после вызова property(my_balance)\
my_balance теперь является property obj. Как мы видили у property obj есть метод setter(). \
Будем теперь этим пользоваться


In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

    def set_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

    my_balance = my_balance.setter(set_balance)


user = BankAccount("Mikhail", 1000)
print(user.my_balance)
user.my_balance = 1234
print(user.my_balance)
user.set_balance = 43
print(user.my_balance)

Замечательно, теперь мы снова столкнулись с двойной функциональностью. Почему так произошло ?\
Ответ: потому что метод set_balance не был задекорирован (set_balance не стало функцией,\
которую возвращает внутри себя декоратор)\
\
Хорошо, попробуем поступить также как и с геттером, для начала назовем сеттер также как и my_balance ...

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

    def my_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

    my_balance = my_balance.setter(set_balance)


Получился конфликт имен: имя свойства совпадает с именем функции (А что ? Разве у нас не было конфликта имени геттера с свойством ?)\
Имя my_balance перезаписалось сеттером и теперь не является объектом property и метод setter недоступен\
\
Хорошо, теперь поступим так: будем сохранять ссылку на задекорированную функцию my_balance в имя my_property_balance. В конечном счете my_balance станет свойством\
(иногда для простоты высказывания я буду называть объект класса property свойством)

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

    my_property_balance = my_balance

    def my_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

    my_balance = my_property_balance.setter(set_balance)

Ура, теперь мы сделали то, что хотели, но остался делитер...\
Делитер можно также сделать с помощью трюка с перезаписью имен. (Написать таким образом делитер остается в качестве упражнения :) \)\
Но зачем так напрягаться, когда у нас есть возможность пользоваться синтаксическим сахаром питона

In [ ]:
class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

    @my_balance.setter
    def my_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value

my_balance это наша задекорированная функция и после @ мы можем у неё вызвать setter !\
Можете проверить, все будет работать\
\
Ну а теперь напишем делитер используя такую штуку

In [ ]:

class BankAccount:

    def __init__(self, name, balance):
        self.name = name
        self.__balance = balance

    @property
    def my_balance(self):
        print("get balance")
        return self.__balance

    @my_balance.setter
    def my_balance(self, value):
        print("set balance")
        if not isinstance(value, (int, float)):
            raise ValueError("Баланс должен быть числом")
        self.__balance = value
    
    @my_balance.deleter
    def my_balance(self):
        print("delete balance")
        del self.__balance

Самое важное, что здесь нужно знать так это вот что: чтобы пользоваться этой вещью названия геттера, сеттера и делитера должны совпадать

Property Вычисляемые свойства (Calculated properties python)\
\
Научились создавать геттеры, сеттеры и делитеры с помощью property, теперь научимся использовать property в качестве вычисляемых атрибутов\
\
Представим что есть проблема\
\
Есть класс, который описывает квадрат. У него есть метод, который вычисляет площадь


In [ ]:
class Square:

    def __init__(self, s):
        self.side = s

    def area(self):
        return self.side ** 2


a = Square(5)
print(a.area())

С одной стороны можно без проблем вычислять площадь квадрата с помощью метода. Но с другой стороны было бы логично, чтобы площадь квадрата была\
атрибутом-переменной класса. То есть мы при инициализации указали бы сторону квадрата и класс автоматически как-нибудь нам бы сделал так, чтобы\
вычислилось значение площади и записалось в атрибут-переменную\
Мы можем использовать property в этих целях

In [ ]:
class Square:

    def __init__(self, s):
        self.side = s
        self.__area = None

    @property
    def area(self):
        if self.__area is None:
            self.__area = self.side ** 2

        return self.__area

Замечательно, но теперь мы столкнулись с ещё одной проблемой. Даже если мы где-либо в коде будем изменять значение side, то значение area никак не будет изменено\
Нам нужно уметь обрабатывать ситуацию, когда будет изменено значение атрибута side.\
Будем использовать уже полученные знания.\
Сделаем side приватной переменной, сделаем её свойством и установим для неё setter и getter

In [ ]:
class Square:

    def __init__(self, s):
        self.__side = s
        self.__area = None

    @property
    def side(self):
        return self.__side

    @side.setter
    def side(self, value):
        self.__side = value
        self.__area = None

    @property
    def area(self):
        if self.__area is None:
            self.__area = self.__side ** 2

        return self.__area

Хорошо, но есть одна маленькая деталька. Свойство area должно выглядеть вот так:

In [ ]:
    @property
    def area(self):
        if self.__area is None:
            self.__area = self.side ** 2

        return self.__area

Поняли почему ? Правильно, мы определили свойство side и теперь можем не обращаться напрямую к self.__side, а вместо этого использовать геттер. \
Деталь то маленькая, но в этом и состоит принцип dry (don't repeat yourself), который заключается в том, чтобы не повторять уже существующие участки кода (не повторять уже реализованный функционал)\
\
Ну вот один из способов задавть вычисляемые свойства, но можно еще по-другому, например используя декоратор cache из библиотеки functools


In [ ]:
from functools import cache

class Square:
    def __init__(self, s):
        self.__side = s
        
    @property
    @cache
    def area(self):
        self.__area = self.__side**2
        return self.__area